In [1]:
# import libraries
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep
import time
from pandas import json_normalize
from sklearn.cluster import KMeans
import numpy as np
from itertools import chain


In [2]:

# Consists of our spotify credentials to log in
secret_file = open("secret.txt","r") #r = opening for reading, needs to be specified

# Read the whole file in one go
string = secret_file.read()

string.split('\n')

# Translate the set of strings in a dictionary
secret_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secret_dict[line.split(':')[0]]=line.split(':')[1].strip()
        

In [3]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secret_dict['clientid'],
                                                           client_secret=secret_dict['clientsecret']))

In [4]:
top100 = pd.read_csv('top_100.csv')
top100

,Name of Song,Artist
0,Flowers,Miley Cyrus
1,Unholy,Sam Smith & Kim Petras
2,Anti-Hero,Taylor Swift
3,wait in the truck,HARDY & Lainey Wilson
4,Hey There Delilah,Plain White T's
...,...,...
95,Build a Boat,Colton Dixon
96,Until I Found You (Em Beihold Version),Stephen Sanchez & Em Beihold
97,World's Smallest Violin,AJR
98,Tennessee Orange,Megan Moroney


In [5]:
top100 = top100.rename(columns={'Name of Song': 'title', 'Artist': 'artist'})
top100

,title,artist
0,Flowers,Miley Cyrus
1,Unholy,Sam Smith & Kim Petras
2,Anti-Hero,Taylor Swift
3,wait in the truck,HARDY & Lainey Wilson
4,Hey There Delilah,Plain White T's
...,...,...
95,Build a Boat,Colton Dixon
96,Until I Found You (Em Beihold Version),Stephen Sanchez & Em Beihold
97,World's Smallest Violin,AJR
98,Tennessee Orange,Megan Moroney


In [6]:
top100.values

array([['Flowers', 'Miley Cyrus'],
       ['Unholy', 'Sam Smith & Kim Petras'],
       ['Anti-Hero', 'Taylor Swift'],
       ['wait in the truck', 'HARDY & Lainey Wilson'],
       ['Hey There Delilah', "Plain White T's"],
       ['Made You Look', 'Meghan Trainor'],
       ['Gangsta Boo', 'Ice Spice & Lil Tjay'],
       ['Lift Me Up (From Black Panther: Wakanda Forever - Music From and Inspired By)',
        'Rihanna'],
       ['Thought You Should Know', 'Morgan Wallen'],
       ['Crazy', 'Gnarls Barkley'],
       ['Heart Like A Truck', 'Lainey Wilson'],
       ["I'm Good (Blue)", 'David Guetta & Bebe Rexha'],
       ['Shakira: Bzrp Music Sessions, Vol. 53', 'Bizarrap & Shakira'],
       ['Still Into You', 'Paramore'],
       ['Unstoppable', 'Sia'],
       ['Thank God', 'Kane Brown & Katelyn Brown'],
       ["I Ain't Worried", 'OneRepublic'],
       ['We Are Young (feat. Janelle Monáe)', 'Fun.'],
       ['Hold My Hand', 'Lady Gaga'],
       ['The Kind of Love We Make', 'Luke Combs'],
  

In [7]:
type(top100.values)

numpy.ndarray

In [8]:
data_recommender = pd.read_csv("data_recommender.csv")
data_recommender.head()

,artist,song,popularity,uri,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,clusters
0,Daphni,Cloudy - Kelbin Remix,0,spotify:track:4OtqragtOuKh41rBNnFXuK,0.758,0.878,11,-6.656,0,0.0549,...,0.1000,0.0668,134.019,audio_features,4OtqragtOuKh41rBNnFXuK,https://api.spotify.com/v1/tracks/4OtqragtOuKh...,https://api.spotify.com/v1/audio-analysis/4Otq...,336716,4,9
1,Skrillex,Rumble,84,spotify:track:1GfBLbAhZUWdseuDqhocmn,0.810,0.836,1,-7.721,0,0.0645,...,0.0598,0.0585,139.979,audio_features,1GfBLbAhZUWdseuDqhocmn,https://api.spotify.com/v1/tracks/1GfBLbAhZUWd...,https://api.spotify.com/v1/audio-analysis/1GfB...,146571,4,7
2,altrice,yoni,31,spotify:track:0KpuGGkI2ELoGm7J8aDQbK,0.863,0.773,11,-3.944,0,0.3400,...,0.1000,0.6620,123.339,audio_features,0KpuGGkI2ELoGm7J8aDQbK,https://api.spotify.com/v1/tracks/0KpuGGkI2ELo...,https://api.spotify.com/v1/audio-analysis/0Kpu...,336230,4,0
3,Junior Boys,Night Walk,23,spotify:track:1ufmBDHamjH3v8Ms1cyNsN,0.652,0.244,5,-21.979,1,0.0540,...,0.1330,0.5220,169.810,audio_features,1ufmBDHamjH3v8Ms1cyNsN,https://api.spotify.com/v1/tracks/1ufmBDHamjH3...,https://api.spotify.com/v1/audio-analysis/1ufm...,289960,4,5
4,Zaza,Nduzangou,0,spotify:track:261oqGIHQWXeBWoLNsnMDm,0.625,0.624,0,-7.812,0,0.1390,...,0.1350,0.7690,119.975,audio_features,261oqGIHQWXeBWoLNsnMDm,https://api.spotify.com/v1/tracks/261oqGIHQWXe...,https://api.spotify.com/v1/audio-analysis/261o...,291273,4,2


In [9]:
recommender = data_recommender[['artist', 'song', 'clusters']]
recommender


,artist,song,clusters
0,Daphni,Cloudy - Kelbin Remix,9
1,Skrillex,Rumble,7
2,altrice,yoni,0
3,Junior Boys,Night Walk,5
4,Zaza,Nduzangou,2
...,...,...,...
1331,Teddy Pendergrass,"The More I Get, the More I Want",4
1332,Todd Edwards,Saved My Life,9
1333,Todd Edwards,Never Far From You,9
1334,Todd Edwards,Dancing For Heaven,9


In [10]:
data_clustering = pd.read_csv("data_clustering.csv")
data_clustering

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.758,0.878,11,-6.656,0.0549,0.39000,0.8660,0.1000,0.0668,134.019,336716,4
1,0.810,0.836,1,-7.721,0.0645,0.05150,0.2290,0.0598,0.0585,139.979,146571,4
2,0.863,0.773,11,-3.944,0.3400,0.43500,0.0599,0.1000,0.6620,123.339,336230,4
3,0.652,0.244,5,-21.979,0.0540,0.83200,0.9040,0.1330,0.5220,169.810,289960,4
4,0.625,0.624,0,-7.812,0.1390,0.30600,0.0000,0.1350,0.7690,119.975,291273,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1331,0.716,0.808,5,-9.491,0.0662,0.33500,0.0000,0.0950,0.9020,121.243,264613,4
1332,0.825,0.911,6,-3.918,0.0522,0.00806,0.7800,0.3270,0.9820,124.949,415730,4
1333,0.894,0.917,6,-3.810,0.0904,0.04590,0.8760,0.0585,0.9270,124.995,400748,4
1334,0.866,0.932,6,-4.320,0.0581,0.00851,0.9090,0.0628,0.9840,131.993,349624,4


In [11]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(data_clustering)
X_prep = transformer.transform(data_clustering)
pd.DataFrame(X_prep,columns=data_clustering.columns).head()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.910303,1.183364,1.479620,0.968697,-0.393671,0.180860,1.362555,-0.536524,-1.866257,0.542755,0.126173,0.215973
1,1.193437,0.991169,-1.273928,0.721244,-0.291107,-0.878509,-0.304725,-0.803666,-1.897930,0.774747,-0.877016,0.215973
2,1.482016,0.702877,1.479620,1.598830,2.652283,0.321692,-0.747326,-0.536524,0.405056,0.127037,0.123609,0.215973
3,0.333145,-1.717863,-0.172509,-2.591600,-0.403287,1.564143,1.462016,-0.317228,-0.129191,1.935916,-0.120508,0.215973
4,0.186133,0.021043,-1.549283,0.700100,0.504837,-0.082026,-0.904108,-0.303937,0.813374,-0.003906,-0.113580,0.215973


In [12]:
kmeans = KMeans(n_clusters=10, random_state=1234)
kmeans.fit(X_prep)

KMeans(n_clusters=10, random_state=1234)

In [13]:
clusters = kmeans.predict(X_prep)

# Check the size of the clusters
pd.Series(clusters).value_counts().sort_index()

0    107
1     92
2    194
3    107
4    237
5    200
6     14
7    135
8     36
9    214
dtype: int64

In [14]:
cluster = pd.DataFrame(clusters)


In [15]:
cluster.columns = ['predict_cluster']

In [16]:
cluster

,predict_cluster
0,9
1,7
2,0
3,5
4,2
...,...
1331,4
1332,9
1333,9
1334,9


In [17]:
# Turn everything into lower cases
top100['title']=top100['title'].str.lower()
top100['artist']=top100['artist'].str.lower()
top100.head(2)

,title,artist
0,flowers,miley cyrus
1,unholy,sam smith & kim petras


In [21]:
artist = input("Please enter the artist : ", ).lower()
title = input("What song are you listening to : ", ).lower()


if title in top100.values:
    print("We can recommend this song for you: ")
    print(str(top100.sample(n=1)).title()) #recommends a random song from the list
    
elif title not in top100.values:
    try:
        
        # Step 1 - Replace q with music[1] or title
        #results = sp.search(q='title', limit = 1)
        
        results = sp.search(q='track:' + (''+ title +'') + ', artist:' + (''+ artist +''), type='track', limit=1)
        
        # Step 2 - Find the uri of that song
        song_uri = results["tracks"]["items"][0]["uri"]

        # Step 3 - Find the uri on spotify
        audio_features = sp.audio_features(song_uri)

        # Step 4 - Get a dataframe
        
        audio_features = pd.DataFrame(list(chain.from_iterable(audio_features)))        
        audio_features = audio_df.drop(['mode', 'id','type','track_href','uri', 'analysis_url'], axis = 1)

        # Step 5 - Scale audio_features
        scale_audio_features = transformer.transform(audio_features)

        # Step 6 - Find the cluster
        cluster = kmeans.predict(scale_audio_features)
        cluster = pd.DataFrame(cluster)
        cluster.columns = ['predict_cluster']

        # Step 7 - Chose a random song from the cluster
        pd.options.display.max_colwidth = 300
        songs = pd.merge(left = cluster,
                         right = recommender,
                         how = 'inner',
                         left_on = 'predict_cluster',
                         right_on = "clusters")
        song_rec = songs[['artist', 'song']].sample(n=1)
        artist = song_rec['artist'].to_string(index=False)
        song = song_rec['song'].to_string(index=False)
        
        print("We can recommend ", song ,'by', artist ,'.')
        
    except:
        print('Please try again.')
        time.sleep(5) 
else:
    print("Sorry we cannot recommend you a song at the moment.")

Please enter the artist : taylor
What song are you listening to : anti-hero
We can recommend this song for you: 
                                     Title                        Artist
96  Until I Found You (Em Beihold Version)  Stephen Sanchez & Em Beihold
